In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.utils import resample
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,LSTM,MaxPooling2D,MaxPooling1D,Convolution1D
from matplotlib import pyplot
from numpy import mean
from numpy import std
import numpy as np
from sklearn.metrics import accuracy_score
import numpy
from numpy import array
from numpy import argmax
from keras.models import load_model
from keras.layers import Dense,Activation , Dropout,GRU, LSTM, Bidirectional, Flatten,Conv1D

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
import pickle
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)
x_train=data[0]
y_train=data[1]
x_test=data[2]
y_test=data[3]

In [ ]:
lstm_output_size=100

In [ ]:
def model_hybrid_1(x_train,y_train):
    cnn = Sequential()
    cnn.add(Convolution1D(48, 3, border_mode="same",activation="relu",input_shape=(42, 1)))
    cnn.add(Convolution1D(48, 3, border_mode="same", activation="relu"))
    cnn.add(MaxPooling1D(pool_length=(2)))
    cnn.add(Convolution1D(16, 3, border_mode="same", activation="relu"))
    cnn.add(Convolution1D(16, 3, border_mode="same", activation="relu"))
    cnn.add(MaxPooling1D(pool_length=(2)))
    cnn.add(LSTM(lstm_output_size))
    cnn.add(Dropout(0.1))
    cnn.add(Dense(5, activation="softmax"))
    cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
    cnn.fit(x_train.reshape(12162, 42,1), y_train.reshape(12162,5), nb_epoch=10,verbose=1)
    cnn.save('hybrid_1.h5')
    return cnn

In [ ]:
def model_hybrid_2(x_train,y_train):
    cnn = Sequential()
    cnn.add(Convolution1D(48, 3, border_mode="same",activation="relu",input_shape=(42, 1)))
    cnn.add(Convolution1D(48, 3, border_mode="same", activation="relu"))
    cnn.add(MaxPooling1D(pool_length=(2)))
    cnn.add(Convolution1D(16, 3, border_mode="same", activation="relu"))
    cnn.add(Convolution1D(16, 3, border_mode="same", activation="relu"))
    cnn.add(MaxPooling1D(pool_length=(2)))
    cnn.add(LSTM(lstm_output_size))
    cnn.add(Dropout(0.1))
    cnn.add(Dense(5, activation="softmax"))
    cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
    cnn.fit(x_train.reshape(12162, 42,1), y_train.reshape(12162,5), nb_epoch=15,verbose=1)
    cnn.save('hybrid_2.h5')
    return cnn

In [ ]:
def model_hybrid_3(x_train,y_train):
    cnn = Sequential()
    cnn.add(Convolution1D(48, 3, border_mode="same",activation="relu",input_shape=(42, 1)))
    cnn.add(Convolution1D(48, 3, border_mode="same", activation="relu"))
    cnn.add(MaxPooling1D(pool_length=(2)))
    cnn.add(Convolution1D(16, 3, border_mode="same", activation="relu"))
    cnn.add(Convolution1D(16, 3, border_mode="same", activation="relu"))
    cnn.add(MaxPooling1D(pool_length=(2)))
    cnn.add(LSTM(lstm_output_size))
    cnn.add(Dropout(0.1))
    cnn.add(Dense(5, activation="softmax"))
    cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
    cnn.fit(x_train.reshape(12162, 42,1), y_train.reshape(12162,5), nb_epoch=12,verbose=1)
    cnn.save('hybrid_3.h5')
    return cnn

In [ ]:
def model_biLSTM(x_train,y_train):
    biLSTM = Sequential()
    biLSTM.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True, input_dim = 42)))
    biLSTM.add(Bidirectional(LSTM(100, activation='relu')))
    biLSTM.add(Dense(5, activation = 'softmax'))
    biLSTM.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    # fit model
    biLSTM.fit(x_train.reshape(12162,1, 42), y_train.reshape(12162,5), epochs=40, verbose=1,batch_size = 50)
    biLSTM.save('biLSTM.h5')
    return biLSTM

In [ ]:
def model_gru(x_train,_y_train):
    gru = Sequential()
    gru.add(GRU(100, activation='relu', return_sequences=True, input_dim = 42))
    gru.add(GRU(100, activation='relu'))
    gru.add(Dense(5, activation = 'softmax'))
    gru.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    # fit model
    gru.fit(x_train.reshape(12162,1, 42), y_train.reshape(12162,5), epochs=30, verbose=1,batch_size = 50)
    return gru

In [ ]:
########################################################################################################
################ You can run this line if you want to retrain otherwise skip this line   ###############
########################################################################################################
hybrid_1=model_hybrid_1(x_train,y_train)
hybrid_2=model_hybrid_2(x_train,y_train)
hybrid_3=model_hybrid_3(x_train,y_train)
gru=model_gru(x_train,y_train)
biLSTM=model_biLSTM(x_train,y_train)

In [ ]:
gru=load_model('gru.h5')
print('>loaded GRU')
biLSTM=load_model('biLSTM.h5')
print('>loaded BiLSTM')
hybrid_1=load_model('hybrid_1.h5')
print('>loaded HYBRID_1')
hybrid_2=load_model('hybrid_2.h5')
print('>loaded HYBRID_2')
hybrid_3=load_model('hybrid_3.h5')
print('>loaded HYBRID_3')

In [ ]:
def ensemble_predict(gru,biLSTM,hybrid_1,hybrid_2,hybrid_3, x_test):
	# make predictions
	yhats = [gru.predict(x_test.reshape(5001,1,42)),
            biLSTM.predict(x_test.reshape(5001,1,42)),
            hybrid_1.predict(x_test.reshape(5001,42,1)),
            hybrid_2.predict(x_test.reshape(5001,42,1)),
            hybrid_3.predict(x_test.reshape(5001,42,1))]
	yhats = array(yhats)
	# sum across ensemble members
	summed = numpy.sum(yhats, axis=0)
	# argmax across classes
	result = argmax(summed, axis=1)
	return result

In [ ]:
res=ensemble_predict(gru,biLSTM,hybrid_1,hybrid_2,hybrid_3, x_test)

In [ ]:
tru=np.array([np.argmax(i) for i in data[3]])

In [ ]:
##########################################################
#### ACCURACY OF OUR MODELS BAGGED    ###################
#########################################################
print("Bagged Ensemble Accuracy : ",accuracy_score(tru,res))

In [ ]:
##########################################################
#### Individual ACCURACIES    ###################
#########################################################

In [ ]:
res_biLSTM=[np.argmax(i) for i in biLSTM.predict(x_test.reshape(5001,1,42))]
print("Bidirection LSTM accuracy : ",accuracy_score(tru,res_biLSTM))

In [ ]:
res_gru=[np.argmax(i) for i in gru.predict(x_test.reshape(5001,1,42))]
print("GRU accuracy : ",accuracy_score(tru,res_gru))

In [ ]:
res_hyb1=[np.argmax(i) for i in hybrid_1.predict(x_test.reshape(5001,42,1))]
print("HYBRID 1 accuracy : ",accuracy_score(tru,res_hyb1))

In [ ]:
res_hyb2=[np.argmax(i) for i in hybrid_2.predict(x_test.reshape(5001,42,1))]
print("HYBRID 2 accuracy : ",accuracy_score(tru,res_hyb2))

In [ ]:
res_hyb3=[np.argmax(i) for i in hybrid_3.predict(x_test.reshape(5001,42,1))]
print("HYBRID 3 accuracy : ",accuracy_score(tru,res_hyb3))